In [37]:
data = None

with open('fas_crse_cat.txt', 'r') as f:
    raw_content = f.read()
    data = f.read().splitlines()

In [29]:
# strip any spaces from all lines
data = [d.strip() for d in data]


# remove header lines
HEADER_LINES = (
    'Faculty of Arts and Sciences Course Catalog Preview',
    'Table of Contents'
)
data = [d for d in data if d not in HEADER_LINES]



# remove the page footers
PAGE_FOOTERS = (
    'HARVARD UNIVERSITY',
    '9/4/2017 0:39 AM',
    '9/24/2017 0:39 AM',
    'HARVARD UNIVERSITY 9/24/2017 0:39 AM'
)

data = [d for d in data if d not in PAGE_FOOTERS]

In [30]:
data[0:100]

['African and African American Studies Subject: Hausa',
 'Hausa AA',
 'Elementary Hausa (126307)',
 'John Mugane',
 '2017 Fall (4 Credits)',
 'Instructor Permissions: None',
 'Schedule: TBD Enrollment Cap: n/a',
 'A study of Hausa a most widely used native language and lingua franca in West Africa at the Elementary level (First year part 1). Contact hours supplemented by digital resources. Emphasis on written expression, reading comprehension, and oral fluency. Students must complete both terms of this course (parts AA and AB) within the same academic year in order to receive full Elementary Hausa course credit.',
 'Course Notes:',
 'Additional Course Attributes:',
 'Hausa AB',
 'Elementary Hausa (205858)',
 'John Mugane',
 '2018 Spring (4 Credits)',
 'Instructor Permissions: None',
 'Schedule: TBD Enrollment Cap: n/a',
 'Languages in the tutorial program are offered when there is demonstrated curricular or academic need on the part of the student, and when suitable instruction can be 

In [31]:
# courses are marked by e.g.

"""
Hausa 101BR
Advanced Hausa II (205870)
John Mugane
"""
# 
# .* \(\d{6}\)

In [32]:
# dump data to file

with open('cleaned_catalog.txt','w') as outfile:
    for item in data:
        outfile.write("%s\n" % item)

In [33]:
import pdfminer

In [34]:
pdfminer

<module 'pdfminer' from '/Users/neel/usr/anaconda2/lib/python2.7/site-packages/pdfminer/__init__.pyc'>

In [36]:
from bs4 import BeautifulSoup

In [41]:
raw_content

with open('fas_crse_cat.html', 'r') as f:
    raw_content = f.read()
    
soup = BeautifulSoup(raw_content, 'html.parser')

In [49]:
# all content will be in div.t
# so extract this
elts = soup.find_all('div', class_='t')

<div class="t m0 x0 h2 y1 ff1 fs0 fc0 sc0 ls0 ws0">HARVARD UNIVERSITY</div>
<div class="t m0 x1 h3 y2 ff1 fs0 fc0 sc0 ls0 ws0">Page <span class="ff2">1</span> of <span class="ff2">3603</span></div>
<div class="t m0 x2 h2 y1 ff1 fs0 fc0 sc0 ls0 ws0">9/4/2017 0:39 AM</div>
<div class="t m0 x0 h4 y3 ff1 fs1 fc0 sc0 ls0 ws0">Faculty of Arts and Sciences</div>
<div class="t m0 x0 h4 y4 ff1 fs1 fc0 sc0 ls1 ws0">Course Catalog Preview</div>
<div class="t m0 x0 h5 y5 ff3 fs2 fc1 sc0 ls2 ws0">Table of Contents</div>
<div class="t m0 x0 h6 y6 ff1 fs3 fc2 sc0 ls0 ws0">African and African American Studies.....................................................................................................................................<span class="_ _0"></span>6</div>
<div class="t m0 x3 h6 y7 ff1 fs3 fc2 sc0 ls0 ws0">Subject: Hausa.....................................................................................................................................................................<sp

In [46]:
lst[0:10]

[<div class="t m0 x0 h2 y1 ff1 fs0 fc0 sc0 ls0 ws0">HARVARD UNIVERSITY</div>,
 <div class="t m0 x1 h3 y2 ff1 fs0 fc0 sc0 ls0 ws0">Page <span class="ff2">1</span> of <span class="ff2">3603</span></div>,
 <div class="t m0 x2 h2 y1 ff1 fs0 fc0 sc0 ls0 ws0">9/4/2017 0:39 AM</div>,
 <div class="t m0 x0 h4 y3 ff1 fs1 fc0 sc0 ls0 ws0">Faculty of Arts and Sciences</div>,
 <div class="t m0 x0 h4 y4 ff1 fs1 fc0 sc0 ls1 ws0">Course Catalog Preview</div>,
 <div class="t m0 x0 h5 y5 ff3 fs2 fc1 sc0 ls2 ws0">Table of Contents</div>,
 <div class="t m0 x0 h6 y6 ff1 fs3 fc2 sc0 ls0 ws0">African and African American Studies.....................................................................................................................................<span class="_ _0"></span>6</div>,
 <div class="t m0 x3 h6 y7 ff1 fs3 fc2 sc0 ls0 ws0">Subject: Hausa.........................................................................................................................................................

In [51]:
headings = soup.find_all('div', class_='fs2')

In [62]:
# the .fs2 will be the title
# following .fs3 will be descriptive strings

groups = []
current_group = None

for i in range(0, 1000):
    elt = elts[i]
    
    if 'fs2' in elt['class']:
        # new group
        current_group = dict()
        groups.append(current_group)
        
        current_group['name'] = elt.string
        current_group['strings'] = []
    elif elt is not None:
        if current_group is None:
            continue
        if elt.string is None:
            continue
        BANNED_STRINGS = (
            'HARVARD UNIVERSITY',
            '9/4/2017 0:39 AM',
            '9/24/2017 0:39 AM',
            'HARVARD UNIVERSITY 9/24/2017 0:39 AM'
        )
        if elt.string in BANNED_STRINGS:
            continue`
        current_group['strings'].append(elt.string)

In [63]:
groups

[{'name': u'Table of Contents',
  'strings': [u'HARVARD UNIVERSITY',
   u'9/4/2017 0:39 AM',
   u'HARVARD UNIVERSITY',
   u'9/4/2017 0:39 AM',
   u'HARVARD UNIVERSITY',
   u'9/4/2017 0:39 AM',
   u'HARVARD UNIVERSITY',
   u'9/4/2017 0:39 AM',
   u'HARVARD UNIVERSITY',
   u'9/4/2017 0:39 AM',
   u'African and African American Studies']},
 {'name': u'Subject: Hausa', 'strings': []},
 {'name': u'Hausa     AA ',
  'strings': [u'John Mugane  ',
   u'2017 Fall (4 Credits)',
   u'Schedule: ',
   u'TBD',
   u'Instructor Permissions:',
   u'None',
   u'Enrollment Cap:',
   u'n/a',
   u'A study of Hausa a most widely used native language and lingua franca in West Africa at the Elementary level (First',
   u'year part 1). Contact hours supplemented by digital resources. Emphasis on written expression, reading ',
   u'comprehension, and oral fluency. Students must complete both terms of this course (parts AA and AB) within the ',
   u'same academic year in order to receive full Elementary Hausa co